# Application of MScausality to the experiment from Talus

In [1]:
# MScausality functions
from MScausality.causal_model.LVM import LVM
from MScausality.data_analysis.normalization import normalize
import MScausality.data_analysis.gene_set as gs
import pyro

# Random other stuff
import pandas as pd
import numpy as np

# Graphs
import networkx as nx
from y0.graph import NxMixedGraph

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/Users/kohler.d/opt/anaconda3/envs/MScausality/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Figure out some new indra queries (to delete)

In [2]:
msstats_model = pd.read_csv("../../../data/Talus/processed_data/model.csv")
msstats_model = gs.parse_protein_name(msstats_model, column_name ="Protein", parse_gene=True)

In [3]:
msstats_model[(msstats_model["Label"] == "DMSO-DbET6") & \
                  (msstats_model["adj.pvalue"] < .05) & \
                  (np.isfinite(msstats_model["log2FC"]))]

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
19890,BRD2,DMSO-DbET6,2.046185,0.114339,17.895836,260.0,0.000000e+00,0.000000e+00,NaN,0.310067,0.0
19935,BRD3,DMSO-DbET6,3.333428,0.126571,26.336522,257.0,0.000000e+00,0.000000e+00,NaN,0.252349,0.0
19980,BRD4,DMSO-DbET6,2.668935,0.101283,26.351317,257.0,0.000000e+00,0.000000e+00,NaN,0.118121,0.0
27900,CEBPZ,DMSO-DbET6,-0.291059,0.074340,-3.915236,260.0,1.154421e-04,3.005139e-02,NaN,0.059060,0.0
37530,CRNL1,DMSO-DbET6,-0.268054,0.069816,-3.839419,260.0,1.549581e-04,3.832114e-02,NaN,0.027517,0.0
41580,DAZP1,DMSO-DbET6,0.617508,0.099537,6.203818,260.0,2.159950e-09,1.780518e-06,NaN,0.286577,0.0
66060,FUBP2,DMSO-DbET6,0.291045,0.077226,3.768747,260.0,2.030526e-04,4.782372e-02,NaN,0.017450,0.0
66105,FUBP3,DMSO-DbET6,0.300410,0.069882,4.298798,260.0,2.430235e-05,8.941156e-03,NaN,0.095302,0.0
72855,GTPB4,DMSO-DbET6,-0.344029,0.087806,-3.918077,260.0,1.141655e-04,3.005139e-02,NaN,0.020134,0.0
75735,HEAT3,DMSO-DbET6,-0.409029,0.100056,-4.087980,260.0,5.804139e-05,1.794204e-02,NaN,0.127517,0.0


In [4]:
msstats_model["log2FC"] = abs(msstats_model["log2FC"])
msstats_model[(msstats_model["Label"] == "DMSO-DbET6") & \
                  (np.isfinite(msstats_model["log2FC"])) & \
                    (msstats_model["DF"] > 100)].sort_values(
                      "log2FC", ascending=False).head(90)#["Protein"].values

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
19935,BRD3,DMSO-DbET6,3.333428,0.126571,26.336522,257.0,0.000000,0.000000,NaN,0.252349,0.0
19980,BRD4,DMSO-DbET6,2.668935,0.101283,26.351317,257.0,0.000000,0.000000,NaN,0.118121,0.0
19890,BRD2,DMSO-DbET6,2.046185,0.114339,17.895836,260.0,0.000000,0.000000,NaN,0.310067,0.0
56835,ENOB,DMSO-DbET6,1.177672,0.390995,3.011984,167.0,0.002999,0.268706,NaN,0.342282,0.0
17595,BAZ2B,DMSO-DbET6,0.986227,0.486065,2.029004,110.0,0.044873,0.999700,NaN,0.738255,0.0
...,...,...,...,...,...,...,...,...,...,...,...
76905,HLX,DMSO-DbET6,0.356225,0.194764,1.829006,114.0,0.070013,0.999700,NaN,0.570470,0.0
47835,DNJB6,DMSO-DbET6,0.355106,0.137410,-2.584274,229.0,0.010379,0.658152,NaN,0.399329,0.0
220995,ZN141,DMSO-DbET6,0.353588,0.164723,2.146556,252.0,0.032784,0.999700,NaN,0.244966,0.0
160155,ROBO1,DMSO-DbET6,0.353133,0.200042,1.765291,134.0,0.079793,0.999700,NaN,0.500000,0.0


In [89]:
msstats_model[(msstats_model["Protein"] == "EGFR") & (msstats_model["Label"] == "DMSO-DbET6")]

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
53730,EGFR,DMSO-DbET6,0.018623,0.134889,-0.138059,221.0,0.89032,0.9997,NaN,0.395973,0.0


In [57]:
2.66*.8*.2

0.42560000000000003

In [94]:
def talus_network():

    graph = nx.DiGraph()

    ## Add edges
    graph.add_edge("BRD4", "YAP1")
    graph.add_edge("EGFR", "BRD4")
    graph.add_edge("EGFR", "YAP1")
    graph.add_edge("YAP1", "STAT3")
    graph.add_edge("STAT3", "HLX")

    graph.add_edge("BRD4", "EZH2")
    graph.add_edge("EZH2", "TXNIP")
    # graph.add_edge("MYC", "TXNIP")
    # graph.add_edge("MYC", "BRD4")

    all_nodes = ["BRD4", "YAP1", "EGFR", "STAT3", "HLX", "EZH2", 
                 "TXNIP", "MYC"]
    obs_nodes = ["BRD4", "YAP1", "EGFR", "STAT3", "HLX", "EZH2", 
                 "TXNIP"]
    
    attrs = {node: (True if node not in obs_nodes and 
                    node != "\\n" else False) for node in all_nodes}
    
    nx.set_node_attributes(graph, attrs, name="hidden")
    # Use y0 to build ADMG
    y0_graph = NxMixedGraph()
    y0_graph = y0_graph.from_latent_variable_dag(graph, "hidden")

    return [graph, y0_graph]

In [95]:
tn = talus_network()

In [115]:
msstats_data_all = pd.read_csv("../../../data/Talus/processed_data/ProteinLevelData.csv")
msstats_data = msstats_data_all[msstats_data_all["GROUP"] != "DbET6"]
msstats_data = msstats_data_all
msstats_data = gs.parse_protein_name(msstats_data, column_name ="Protein", parse_gene=True)
input = msstats_data.pivot_table(index="RUN", 
                              columns="Protein", 
                              values="LogIntensities", aggfunc='mean')

In [116]:
# Basic Bayesian model
pyro.clear_param_store()
transformed_data = normalize(input, wide_format=True)
input_data = transformed_data["df"]
scale_metrics = transformed_data["adj_metrics"]
input_data.reset_index(drop=True, inplace=True)

# Full imp Bayesian model
lvm = LVM(backend="numpyro")
lvm.fit(input_data, tn[1])


/Users/kohler.d/Library/CloudStorage/OneDrive-NortheasternUniversity/Northeastern/Research/Causal_Inference/MS_causal_inference/MScausality/src/MScausality/data_analysis/normalization.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: (x - mean) / std)
Compiling.. :   0%|          | 0/2000 [00:00<?, ?it/s]







Running chain 0:   0%|          | 0/2000 [00:17<?, ?it/s]


Running chain 0:  10%|█         | 200/2000 [00:18<00:05, 340.94it/s]


Running chain 0:  20%|██        | 400/2000 [00:18<00:02, 632.97it/s]


Running chain 0:  30%|███       | 600/2000 [00:18<00:01, 825.37it/s]


Running chain 0:  40%|████      | 800/2000 [00:19<00:01, 999.08it/s]


Running chain 0:  50%|█████     | 1000/2000 [00:19<00:00, 1098.60it/s]


Running chain 0:  60%|██████    | 1200/2000 [00:19<00:00, 1066.77it/s]


Running chain 0:  70%|███████   | 1400/2000 [00:19<00:00, 1071.53it/s]


Running chain 0:  80%|████████  | 1600/2000 [00:19<00:0

In [117]:
int1 = {"BRD4": 0}
int2 = {"BRD4": 2.67}
outcome = "HLX"

## MScausality results
lvm.intervention({list(int1.keys())[0]: (list(int1.values())[0] \
                                        - scale_metrics["mean"]) \
                                        / scale_metrics["std"]}, outcome)
mscausality_int_low = lvm.intervention_samples
lvm.intervention({list(int2.keys())[0]: (list(int2.values())[0] \
                                        - scale_metrics["mean"]) \
                                        / scale_metrics["std"]}, outcome)
mscausality_int_high = lvm.intervention_samples

mscausality_int_low = ((mscausality_int_low*scale_metrics["std"]) \
                + scale_metrics["mean"])
mscausality_int_high = ((mscausality_int_high*scale_metrics["std"]) \
                        + scale_metrics["mean"])
mscausality_ate = mscausality_int_high.mean() - mscausality_int_low.mean()
mscausality_ate

Array(0.00638819, dtype=float32)

In [118]:
for key, val in lvm.learned_params.items():
    if "coef" in key:
        print(key, val)

BRD4_EGFR_coef 0.07987555116415024
BRD4_EGFR_coef_scale 0.07408703863620758
EZH2_BRD4_coef 0.10107588768005371
EZH2_BRD4_coef_scale 0.022390050813555717
HLX_STAT3_coef 0.4741003215312958
HLX_STAT3_coef_scale 0.11378706246614456
STAT3_YAP1_coef -0.015296787023544312
STAT3_YAP1_coef_scale 0.05831434205174446
TXNIP_EZH2_coef -0.36267757415771484
TXNIP_EZH2_coef_scale 0.1487201601266861
YAP1_BRD4_coef -0.3122243881225586
YAP1_BRD4_coef_scale 0.15226668119430542
YAP1_EGFR_coef -0.3781295418739319
YAP1_EGFR_coef_scale 0.16137109696865082
